In [1]:
import requests
import json
import datetime as dt
from hdfs import InsecureClient
from bs4 import BeautifulSoup
import pandas as pd
client = InsecureClient('http://localhost:9870', user='big')

In [2]:
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

In [3]:

data=[]

In [6]:
file_dir = '/themapark/dust/'
file_name = 'dust' + cal_std_day(1) + '.json'
base_url=['4129000000','1171000000','4146100000','4211000000','1121500000']
cols=['지역','날짜','미세먼지','초미세먼지']
rows_list = []
for l in base_url:
    url = 'https://www.kr-weathernews.com/mv3/if/pm.fcgi?region={}'.format(l)
    response = requests.get(url)
    res = response.json()
    rows = []
    day = cal_std_day(0)
    #지역
    if l == '4129000000':
        rows.append('과천')
    elif l =='1171000000':
        rows.append('송파')
    elif l =='4146100000':
        rows.append('용인')
    elif l =='4211000000':
        rows.append('춘천')
    elif l =='1121500000':
        rows.append('광진')        
    #날짜
    rows.append(day)
    #미세먼지
    rows.append(res['pm']['history'][-1]['pm10'])
    #초미세먼지
    rows.append(res['pm']['history'][-1]['pm25'])
    rows_list.append(rows)
    for i in range(0,10):
        rows = []
        #지역
        if l == '4129000000':
            rows.append('과천')
        elif l =='1171000000':
            rows.append('송파')
        elif l =='4146100000':
            rows.append('용인')
        elif l =='4211000000':
            rows.append('춘천')
        elif l =='1121500000':
            rows.append('광진')    

        #날짜
        rows.append(str(res['pm']['forcast']['daily'][i]['year']) +'-'+ str(res['pm']['forcast']['daily'][i]['mon']) +'-' + str(res['pm']['forcast']['daily'][i]['day']))
#         #년
#         rows.append(res['pm']['forcast']['daily'][i]['year'])
#         #월
#         rows.append(res['pm']['forcast']['daily'][i]['mon'])
#         #일
#         rows.append(res['pm']['forcast']['daily'][i]['day'])
        #미세먼지
        rows.append(res['pm']['forcast']['daily'][i]['pm10'])
        #초미세먼지
        rows.append(res['pm']['forcast']['daily'][i]['pm25'])
        rows_list.append(rows)
        
for rows in rows_list:
    tmp=dict(zip(cols,rows))
    data.append(tmp)
    
result = {
    'meta':{
        'desc':'미세먼지',
        'cols':{
            '지역':'지역',
            '날짜':'날짜',
            '미세먼지':'미세먼지',
            '초미세먼지':'초미세먼지'
        },
    },
   'data':data
}
# client.write(file_dir+file_name, json.dumps(data, ensure_ascii=False), encoding='utf-8')
# data.write.csv("hdfs:/themapark/dust/")
data.coalesce(1).\
    write.\
    format("com.databricks.spark.csv").\
    option("header", "true").\
    save("/data/output.csv")

AttributeError: 'list' object has no attribute 'coalesce'

In [11]:
type(int(res['pm']['history'][-1]['pm10']))

int

In [9]:
type(res['pm']['forcast']['daily'][i]['pm10'])

int

In [7]:
res['pm']['forcast']['daily'][i]['year']

2022